# 讀取字典

In [1]:
import pandas as pd
import os

In [2]:
basePath = '/Volumes/backup_128G/z_repository/Yang_data/20190720'
dataPath = 'data'

experimentPrePath = '{0}/{1}/實驗組校對y0719/實驗組_前測(experiental-pretest)'.format(basePath, dataPath)
experimentPostPath = '{0}/{1}/實驗組校對y0719/實驗組_後測(experiental-posttest)'.format(basePath, dataPath)
comparisonPrePath = '{0}/{1}/對照組校對y0719new/對照組_前測(control-pretest)'.format(basePath, dataPath)
comparisonPostPath = '{0}/{1}/對照組校對y0719new/對照組_後測(control-posttest)'.format(basePath, dataPath)

file_dic = 'permantData/7000Words_20190512_v2.xlsx'
to_exppre_doc = 'output/experiment_pre_20190720_v2.xlsx'
to_exppost_doc = 'output/experiment_post_20190720_v2.xlsx'
to_compre_doc = 'output/comparison_pre_20190720_v2.xlsx'
to_compost_doc = 'output/comparison_post_20190720_v2.xlsx'

to_level_doc = 'output/level_20190720_v2.xlsx'

write_exppre_doc = '{0}/{1}'.format(basePath, to_exppre_doc)
write_exppost_doc = '{0}/{1}'.format(basePath, to_exppost_doc)
write_compre_doc = '{0}/{1}'.format(basePath, to_compre_doc)
write_compost_doc = '{0}/{1}'.format(basePath, to_compost_doc)

write_level_doc = '{0}/{1}'.format(basePath, to_level_doc)
read_dic = '{0}/{1}'.format(basePath, file_dic)

In [3]:
dicDf = pd.DataFrame()

with pd.ExcelFile(read_dic) as reader:
    # read sheet by sheet
    for sheet in reader.sheet_names:
#         print(sheet)
        sheetDf = pd.read_excel(reader, sheet, header=None)
        sheetDf = sheetDf.fillna(0)

        dicDf = dicDf.append(sheetDf, ignore_index=True)

# change to lowercase
dicDf[0] = dicDf[0].str.lower()
len(dicDf.index)

6813

In [4]:
dicDf.head()

,0,1,2
0,a,art.,1
1,an,art.,1
2,abandon,v.,4
3,abbreviate,v.,6
4,abbreviation,n.,6


# 將英文字 lemmatize（詞形還原）

In [5]:
# 詞性還原 
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
wtlem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def findWord(word, dicDataF):
    found = dicDataF.loc[dicDataF[0] == word]
    if found.empty == False:
        return found.iloc[0][2]
    return None
    
def returnWord(word, level):
    return {'lemma':word, 'level':level}

def lemmatizer(word, dicDataF):
    lowerWord = word.lower()
    
    found = findWord(lowerWord, dicDataF)
    if found != None:
        return returnWord(word, found)
    
    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.ADJ)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.VERB)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)
        
    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.NOUN)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    lemmaWord = wtlem.lemmatize(lowerWord, wordnet.ADV)
    if lemmaWord != lowerWord:
        found = findWord(lemmaWord, dicDataF)
        if found != None:
            return returnWord(lemmaWord, found)

    return returnWord(word, 9)

# 讀檔；去掉非英文字，回傳 word 字串，包含引用次數

In [6]:
# pip install python-docx 
import docx

def isAlpha(word):
    try:
        return word.encode('ascii').isalpha()
    except UnicodeEncodeError:
        return False
    
def wordCount(wordList):
    wCount = {}
    for word in wordList:
        if word not in wCount:
            wCount[word] = 1
        else:
            wCount[word] += 1
    return wCount
    
def split2Words(txt):
    wordList = []
    cleanTxt = txt.replace(',', '').replace('.', '')
    splitWords = cleanTxt.split(' ')
    for word in splitWords:
        if '’' in word or '-' in word:
            wordList.append(word)
            continue

        if isAlpha(word) == True:
            wordList.append(word)
            
    return wordCount(wordList)
    
def replaceMultiple(mainString, toBeReplaces, newString):
#     outString = copy.copy(mainString)
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return mainString

def readtxt(filename):
    fullText = []
    doc = docx.Document(filename)
    tables = doc.tables
    for table in tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    processedPara = replaceMultiple(paragraph.text, ['…', '’', '‘', '-'] , ' ')
#                     print(processedPara)
                    fullText.append(processedPara)
#                     print(paragraph.text)

    fullText = ' '.join(fullText)
    return split2Words(fullText)

# 讀檔列表

In [7]:
def processDoc(wordList):
    resWordList = {}
    for word in wordList:
        lowerWord = word.lower()
        worddic = lemmatizer(lowerWord, dicDf)
        if worddic['lemma'] not in resWordList:
            resWordList[worddic['lemma']] = {'words':word, 'count':wordList[word], 
                                             'level':worddic['level'], 
                                             'index':'{0}({1})'.format(worddic['lemma'], wordList[word])}
        else:
            resWordList[worddic['lemma']]['words'] += ';{0}'.format(word)
            resWordList[worddic['lemma']]['count'] += wordList[word]
            resWordList[worddic['lemma']]['index'] = '{0}({1})'.format(worddic['lemma'], resWordList[worddic['lemma']]['count'])
            if resWordList[worddic['lemma']]['level'] != worddic['level']:
                print("ERROR!!", word, worddic)

    #     print(word, lemmatizer(word, dicDf))
    return resWordList

# Parse Directory

In [8]:
def parseDir(filedir, outputFile):
    levelColumns = []
    levelNums = []
    levelWords = []
    with pd.ExcelWriter(outputFile) as writer:
        for root, dirs, files in os.walk(filedir):
            for file in sorted(files):
                if '.docx' in file and file[0] != '.':
                    # The file could be temporary opened.
                    if file[0] == '~':
                        continue

                    sheetName = file.replace('.docx', '')

                    filename = os.path.join(root, file)
                    print(filename)
                    wordList = readtxt(filename)
                    processedList = processDoc(wordList)

                    # output
                    outDf = pd.DataFrame(processedList).T
                    outDf.sort_values(by=['level', 'words'], inplace=True)
#                     print(outDf.head())

                    # list level words
                    levelColumns.append(sheetName)

                    numWords = {}
                    wordWords = {}
#                     for level in range(1, 7):
                    for level in [0, 1, 2, 3, 4, 5, 6, 9]:
                        words = outDf.loc[outDf['level'] == level].index
                        count = outDf.loc[outDf['level'] == level]['count'].sum()
#                         print(level, count)
#                         numWords[level] = len(words)
                        numWords[level] = count
                        wordWords[level] = ';'.join(words)

                    levelNums.append(numWords)
                    levelWords.append(wordWords)
                    
                    # reindex
                    outDf.set_index('index', inplace=True)
                    
                    # write file
                    outDf.to_excel(writer, sheetName)
                    writer.save()

    levelNumDf = pd.DataFrame(levelNums, index=levelColumns)
    levelWordsDf = pd.DataFrame(levelWords, index=levelColumns)
    return levelNumDf, levelWordsDf

In [9]:
resMap = []

# Comparison Pretest
numDf, wordsDf = parseDir(comparisonPrePath, write_compre_doc)
resObj = {'name':'COMP-Pre Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'COMP-Pre Words', 'data':wordsDf}
resMap.append(resObj)

# Comparison Posttest
numDf, wordsDf = parseDir(comparisonPostPath, write_compost_doc)
resObj = {'name':'COMP-Post Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'COMP-Post Words', 'data':wordsDf}
resMap.append(resObj)

# Experimental Pretest
numDf, wordsDf = parseDir(experimentPrePath, write_exppre_doc)
resObj = {'name':'EXP-Pre Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'EXP-Pre Words', 'data':wordsDf}
resMap.append(resObj)

# Experimental Posttest
numDf, wordsDf = parseDir(experimentPostPath, write_exppost_doc)
resObj = {'name':'EXP-Post Num', 'data':numDf}
resMap.append(resObj)

resObj = {'name':'EXP-Post Words', 'data':wordsDf}
resMap.append(resObj)


/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT01-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT02-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT03-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT04-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT05-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT06-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT07-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT09-pre.docx
/Volumes/backup_128G/z_repository/Yang_data/20190720/data/對照組校對y0719new/對照組_前測(control-pretest)/CONT11-p

# Write level words/nums

In [10]:
with pd.ExcelWriter(write_level_doc) as writer:
    # write file
    for row in resMap:
        name = row['name']
        data = row['data']
        
        data.to_excel(writer, name)
    
    writer.save()